# OrientDB tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [OrientDB SQL reference](http://www.orientdb.com/docs/last/SQL-Functions.html)
* [Orientdb python client](http://orientdb.com/docs/last/PyOrient-Client.html#working-with-the-client)

## Import libraries

In [ ]:
import pyorient

In [ ]:
ROOT_PASSWORD = "root"
client = pyorient.OrientDB("localhost", 2424)
try:
    session_id = client.connect("root", ROOT_PASSWORD)
    print("Connected to OrientDB server successfully.")
except pyorient.PyOrientException as e:
    print(f"Failed to connect to OrientDB server: {e}")

In [ ]:
print(client.db_list())

## I. Quick start

### Creating the database

**Q:** Create a database `gods` as a `GRAPH_DATABASE` in `MEMORY_STORAGE_TYPE`. 

We will use it to store relationships between Greek deities.

In [ ]:
try:
    client.db_create("gods", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)
    print("Database 'gods' created successfully.")
except pyorient.PyOrientException as e:
    print(f"Failed to create database: {e}")

**Q:** Connect your pyorient client to the `gods` database.

In [ ]:
try:
    client.db_open("gods", "root", ROOT_PASSWORD)
    print("Connected to 'gods' database successfully.")
except pyorient.PyOrientException as e:
    print(f"Failed to connect to database: {e}")

**Q:** You should now be able to launch OrientDB queries through the Python client with the [command()](http://orientdb.com/docs/last/PyOrient-Client-Command.html) function. 

You should think of OrientDB as a Graph-Document database for the following questions. Each vertex and edge will contain information on it inside a JSON document.

Create a new Vertex with content `{name: 'Zeus', symbol: 'thunder'}`. The [CREATE VERTEX : Create a vertex using JSON content](http://orientdb.com/docs/last/SQL-Create-Vertex.html) doc page should help you.

In [ ]:
try:
    client.command("CREATE VERTEX SET name = 'Zeus', symbol = 'thunder'")
    print("Vertex for Zeus created successfully.")
except pyorient.PyOrientException as e:
    print(f"Failed to create vertex: {e}")

You have created a VERTEX in the previous question. The VERTEX is a [class](https://orientdb.com/docs/last/Tutorial-Classes.html) of OrientDB which defines a record that can be linked to others through EDGE instances.

You can find all VERTEX created in the database with a SQL command on the `V` table, like `SELECT * FROM V`. 

**Q:** Print all current vertices in `gods`, it should only have `Zeus` though for now.

In [ ]:
try:
    vertices = client.command("SELECT * FROM V")
    for vertex in vertices:
        print(vertex)
except pyorient.PyOrientException as e:
    print(f"Failed to query vertices: {e}")

**Q:** Create new vertices with content : 
```
{name:Héra, symbol:tiara}
{name:Poséidon, symbol:trident}
{name:Athena, symbol:helmet}
{name:Arès, symbol:weapons} 
```

In [ ]:
try:
    client.command("CREATE VERTEX SET name = 'Héra', symbol = 'tiara'")
    client.command("CREATE VERTEX SET name = 'Poséidon', symbol = 'trident'")
    client.command("CREATE VERTEX SET name = 'Athena', symbol = 'helmet'")
    client.command("CREATE VERTEX SET name = 'Arès', symbol = 'weapons'")
    print("Vertices created successfully.")
except pyorient.PyOrientException as e:
    print(f"Failed to create vertices: {e}")

**Q:** Display all vertices with name = `Arès`

In [ ]:
try:
    ares_vertices = client.command("SELECT * FROM V WHERE name = 'Arès'")
    for vertex in ares_vertices:
        print(vertex)
except pyorient.PyOrientException as e:
    print(f"Failed to query Arès vertices: {e}")

**Q:** Create an EDGE from `Zeus` to `Poséidon` with the content `{kind: 'sibling'}

In [ ]:
try:
    client.command("""
        CREATE EDGE FROM 
        (SELECT FROM V WHERE name = 'Zeus') 
        TO 
        (SELECT FROM V WHERE name = 'Poséidon') 
        SET kind = 'sibling'
    """)
    print("Edge created successfully between Zeus and Poséidon")
except pyorient.PyOrientException as e:
    print(f"Failed to create edge: {e}")

**Q:** Redisplay all vertices, discuss.

In [ ]:
try:
    vertices = client.command("SELECT * FROM V")
    print("All vertices in the database:")
    for vertex in vertices:
        print(f"\nVertex: {vertex.name}")
        print(f"Symbol: {vertex.symbol}")
        # Display the edge information if it exists
        if hasattr(vertex, 'out_'):
            print("Outgoing edges:", vertex.out_)
        if hasattr(vertex, 'in_'):
            print("Incoming edges:", vertex.in_)
except pyorient.PyOrientException as e:
    print(f"Failed to query vertices: {e}")

**Q:** Display all edges. They are contained in the class `E`

In [ ]:
try:
    edges = client.command("SELECT * FROM E")
    print("All edges in the database:")
    for edge in edges:
        print(f"\nEdge: {edge.kind}")
        print(f"From vertex: {edge.out}")
        print(f"To vertex: {edge.in_}")
except pyorient.PyOrientException as e:
    print(f"Failed to query edges: {e}")

Two fields on vertices have appeared, containing the outgoing (out_) and incoming (in_) links.

At the edge level, two fields point to the original (out) and destination (in) vertices.

**Q:** Lets create some more edges :

* Zeus > Héra (sibling)
* Zeus > Arès (father)
* Zeus > Athena (father)
* Héra > Arès (mother)
* Héra > Zeus (sibling)
* Poséidon > Zeus (sibling)

_Hint 1 :_ check [the CREATE EDGE doc page](http://orientdb.com/docs/last/SQL-Create-Edge.html) to find an example for creating edges on vertices using subqueries so you can run queries to fetch the vertices before creating an edge in between.

_Hint 2 :_ after you have found the command to create edges between vertices with sub-queries, you should be well-versed enough in Python to create a list of all edges in the question, and loop the command on each element of the list to create all edges in one go =)

In [ ]:
# Define the relationships as a list of tuples (from_god, to_god, relationship)
edges = [
    ('Zeus', 'Héra', 'sibling'),
    ('Zeus', 'Arès', 'father'),
    ('Zeus', 'Athena', 'father'),
    ('Héra', 'Arès', 'mother'),
    ('Héra', 'Zeus', 'sibling'),
    ('Poséidon', 'Zeus', 'sibling')
]

try:
    for from_god, to_god, rel in edges:
        client.command(f"""
            CREATE EDGE FROM 
            (SELECT FROM V WHERE name = '{from_god}')
            TO
            (SELECT FROM V WHERE name = '{to_god}')
            SET kind = '{rel}'
        """)
    print("All edges created successfully")
except pyorient.PyOrientException as e:
    print(f"Failed to create edges: {e}")

### Looking for data

**Q:** Using [out()](http://orientdb.com/docs/last/Tutorial-Working-with-graphs.html#querying-graphs) function, display all vertices connected and outgoing from Zeus.

You should use the EXPAND() special function to transform the vertex collection in the result-set by expanding it, making the results more readable.

In [ ]:
try:
    zeus_connections = client.command("SELECT EXPAND(out()) FROM V WHERE name = 'Zeus'")
    print("Vertices connected to Zeus through outgoing edges:")
    for vertex in zeus_connections:
        print(f"\nVertex: {vertex.name}")
        print(f"Symbol: {vertex.symbol}")
except pyorient.PyOrientException as e:
    print(f"Failed to query Zeus connections: {e}")

**Q:** Display all vertices which got a father (the vertices which are the destination of an arc whose kind attribute is father).

_Hint: You can notice that we use the field `in` the arc, and not the function `in()` which applies to vertices._

In [ ]:
try:
    vertices_with_father = client.command("SELECT FROM V WHERE @rid IN (SELECT in FROM E WHERE kind = 'father')")
    print("Vertices that have a father:")
    for vertex in vertices_with_father:
        print(f"\nVertex: {vertex.name}")
        print(f"Symbol: {vertex.symbol}")
except pyorient.PyOrientException as e:
    print(f"Failed to query vertices with father: {e}")

**Q:** As in SQL, the operator `in` used in a clause `where` allows to restrict the possible values with an embedded query _(where ... in (select ...))_. 

Display the mothers, by displaying the vertices where an outgoing arc is part of the arcs where kind is a mother.

In [ ]:
try:
    mothers = client.command("SELECT FROM V WHERE @rid IN (SELECT out FROM E WHERE kind = 'mother')")
    print("Vertices that are mothers:")
    for vertex in mothers:
        print(f"\nVertex: {vertex.name}")
        print(f"Symbol: {vertex.symbol}")
except pyorient.PyOrientException as e:
    print(f"Failed to query mothers: {e}")

**Q:** Display the brothers and sisters of Zeus (the destination summits of an arc whose kind is sibling and whose original summit is Zeus).

In [ ]:
try:
    zeus_siblings = client.command("SELECT EXPAND(out('E[kind=sibling]')) FROM V WHERE name = 'Zeus'")
    print("Zeus's siblings:")
    for vertex in zeus_siblings:
        print(f"\nVertex: {vertex.name}")
        print(f"Symbol: {vertex.symbol}")
except pyorient.PyOrientException as e:
    print(f"Failed to query Zeus's siblings: {e}")

## Modeling a Product Recommendation System

You are currently modeling the data of a product recommendation system with OrientDB.

The main purpose of such a system is to answer the question "which products were purchased by their people who purchased product X? »

Purchased products have only one name field. They are purchased by people who have a nickname.

When a person buys a product, the date of purchase is stored. 

Instead of working with "anonymous" vertices and arcs, you will use classes. The `create class` command allows you to create custom classes.

The vertex classes must extend V, the arc classes must extend E.

**Q:** Create an `eCommerce` database, and the necessary classes to model the system.

PS : you can view all classes in the database with :

```python
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)
```

In [ ]:
# Create eCommerce database
try:
    client.db_create("eCommerce", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)
    print("Database 'eCommerce' created successfully.")
except pyorient.PyOrientException as e:
    print(f"Failed to create database: {e}")

# Connect to the new database
try:
    client.db_open("eCommerce", "root", ROOT_PASSWORD)
    print("Connected to 'eCommerce' database successfully.")
except pyorient.PyOrientException as e:
    print(f"Failed to connect to database: {e}")

# Create classes for products and people (vertices)
try:
    client.command("CREATE CLASS Product EXTENDS V")
    client.command("CREATE CLASS Person EXTENDS V")
    print("Vertex classes created successfully.")
except pyorient.PyOrientException as e:
    print(f"Failed to create vertex classes: {e}")

# Create class for purchases (edges) with date property
try:
    client.command("CREATE CLASS Purchase EXTENDS E")
    client.command("CREATE PROPERTY Purchase.date DATE")
    print("Edge class created successfully.")
except pyorient.PyOrientException as e:
    print(f"Failed to create edge class: {e}")

# Display all classes
print("\nClasses in the database:")
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name.name)

**Q:** Create the following products: `spaghetti`, `bolognese sauce`, `cheese`, `apple`.

In [ ]:
try:
    # Create products
    products = ['spaghetti', 'bolognese sauce', 'cheese', 'apple']
    for product in products:
        client.command(f"CREATE VERTEX Product SET name = '{product}'")
    print("Products created successfully.")
except pyorient.PyOrientException as e:
    print(f"Failed to create products: {e}")

**Q:** Create the following people: `peter`, `meredith`.

In [ ]:
try:
    people = ['peter', 'meredith']
    for person in people:
        client.command(f"CREATE VERTEX Person SET nickname = '{person}'")
    print("People created successfully.")
except pyorient.PyOrientException as e:
    print(f"Failed to create people: {e}")

**Q:** Create the following purchases: 
- peter > spaghetti + cheese on 20/01/2016 
- meredith > cheese + apple + bolognese sauce on 22/01/2016
- peter > spaghetti + bolognese sauce on 27/01/2016


In [ ]:
try:
    # Create purchases for Peter on 20/01/2016
    client.command("""
        CREATE EDGE Purchase FROM 
        (SELECT FROM Person WHERE nickname = 'peter')
        TO
        (SELECT FROM Product WHERE name = 'spaghetti')
        SET date = '2016-01-20'
    """)
    client.command("""
        CREATE EDGE Purchase FROM 
        (SELECT FROM Person WHERE nickname = 'peter')
        TO
        (SELECT FROM Product WHERE name = 'cheese')
        SET date = '2016-01-20'
    """)

    # Create purchases for Meredith on 22/01/2016
    client.command("""
        CREATE EDGE Purchase FROM 
        (SELECT FROM Person WHERE nickname = 'meredith')
        TO
        (SELECT FROM Product WHERE name = 'cheese')
        SET date = '2016-01-22'
    """)
    client.command("""
        CREATE EDGE Purchase FROM 
        (SELECT FROM Person WHERE nickname = 'meredith')
        TO
        (SELECT FROM Product WHERE name = 'apple')
        SET date = '2016-01-22'
    """)
    client.command("""
        CREATE EDGE Purchase FROM 
        (SELECT FROM Person WHERE nickname = 'meredith')
        TO
        (SELECT FROM Product WHERE name = 'bolognese sauce')
        SET date = '2016-01-22'
    """)

    # Create purchases for Peter on 27/01/2016
    client.command("""
        CREATE EDGE Purchase FROM 
        (SELECT FROM Person WHERE nickname = 'peter')
        TO
        (SELECT FROM Product WHERE name = 'spaghetti')
        SET date = '2016-01-27'
    """)
    client.command("""
        CREATE EDGE Purchase FROM 
        (SELECT FROM Person WHERE nickname = 'peter')
        TO
        (SELECT FROM Product WHERE name = 'bolognese sauce')
        SET date = '2016-01-27'
    """)
    
    print("All purchases created successfully.")
except pyorient.PyOrientException as e:
    print(f"Failed to create purchases: {e}")

**Q:** Who bought Bolognese sauce?

In [ ]:
try:
    buyers = client.command("""
        SELECT EXPAND(in('Purchase')) 
        FROM Product 
        WHERE name = 'bolognese sauce'
    """)
    print("People who bought Bolognese sauce:")
    for buyer in buyers:
        print(f"- {buyer.nickname}")
except pyorient.PyOrientException as e:
    print(f"Failed to query buyers: {e}")

**Q:** It is possible to link the `out` and `in` navigation functions. What products are purchased with Bolognese sauce? 

In [ ]:
try:
    # Find products purchased by people who bought bolognese sauce
    co_purchased = client.command("""
        SELECT EXPAND(out('Purchase')) 
        FROM Person 
        WHERE @rid IN (
            SELECT in FROM Purchase 
            WHERE out IN (
                SELECT FROM Product 
                WHERE name = 'bolognese sauce'
            )
        )
        AND @rid IN (
            SELECT in FROM Purchase 
            WHERE out.name <> 'bolognese sauce'
        )
    """)
    
    print("Products purchased by people who bought Bolognese sauce:")
    for product in co_purchased:
        print(f"- {product.name}")
except pyorient.PyOrientException as e:
    print(f"Failed to query co-purchased products: {e}")

## Postquisites

Since we create databases in memory, they get destroyed on server shutdown.